In [ ]:
install.packages("devtools")
install.packages("BiocManager")
BiocManager::install("GEOquery")
devtools::install_github("mkuhn/dict")
install.packages("sjmisc")

In [1]:
library(data.table)
library(magrittr)
library(dplyr)
library(GEOquery)
library(sjmisc)
library(dict)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:data.table’:

    between, first, last

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicat

In [2]:
### LOADING txt files
# read cmv lab test results
fn = "lab_test.txt"
lab_test = fread(fn, data.table = F)
cmv = lab_test%>%filter(NAME_REPORTED=="CMV Ab")
#View(cmv)

# read expsample to biosample
fn = "expsample_2_biosample.txt"
e2b = fread(fn, data.table = F)


# read biosample table
fn = "biosample.txt"
biosample = fread(fn, data.table = F)
biosample = biosample%>%select(SUBJECT_ACCESSION, BIOSAMPLE_ACCESSION, STUDY_ACCESSION)


# find gene expression result
fn = "expsample_public_repository.txt"
pub_repos = fread(fn, data.table = F)
pub_repos = pub_repos%>%filter(REPOSITORY_NAME=="GEO")


### JOINING TABLES
#(1) map CMV result to subject ID
cmv = cmv%>%inner_join(biosample, by="BIOSAMPLE_ACCESSION")


#(2) map gene expression result to biosample w/ expsample_accession
pub_repos1 = pub_repos%>%inner_join(e2b[, c("EXPSAMPLE_ACCESSION","BIOSAMPLE_ACCESSION")], by="EXPSAMPLE_ACCESSION")


#(3) map gene expression result to subject
pub_repos2 = pub_repos1%>%inner_join(biosample, by="BIOSAMPLE_ACCESSION")


# map gene expression with CMV result
results = cmv%>%inner_join(pub_repos2, by=c("SUBJECT_ACCESSION","STUDY_ACCESSION"))
#print(head(results))
#dim(results)

In [ ]:
count = 0
gpl_dict = dict()
nonfunc_gpl = dict()### non-functional GPL ID dict or list for colnames
nonfunc_gsm = vector()

In [ ]:
for(i in results$REPOSITORY_ACCESSION) {
  ### LOADING TABLES
  GSM = getGEO(i, getGPL=FALSE)
    
  if(length(colnames(GSM@dataTable@table)) == 0){
      nonfunc_gsm = c(nonfunc_gsm, i)
      next
  }  
    
  genex = GSM@dataTable@table[,1:2]# it exists, load it
  platform_char = GSM@header$platform_id
  
  ### check if platform id already loaded
  if(platform_char %in% gpl_dict$keys()){
    ### at this point the platform table should only have cols ID and Symbol
    gene_profiles = genex%>%inner_join(gpl_dict[[platform_char]], by=c("ID_REF"="ID"))
    
  } else { ### we have a unique platform GPL ID to add to the dict
    ### load unique GPL
    platform_obj = getGEO(platform_char, getGPL=FALSE)
    platform = platform_obj@dataTable@table
    
    if("ID" %in% colnames(platform) && str_contains(colnames(platform), "Symbol")){
        names(platform)[grepl("Symbol", names(platform))] = "Symbol"
        platform = platform[,c("ID","Symbol")]
    
        gpl_dict[[platform_char]] = platform ## add platform table to dict
        gene_profiles = genex%>%inner_join(platform, by=c("ID_REF"="ID"))
      
    } else {
        ### store non-functional GPL w/ platform table or colnames in dict
        #nonfunc_gpl[[platform_char]] = c(nonfunc_gpl[[platform_char]], platform)# should create a list for each key
        #nonfunc_gpl[[platform_char]] = append(dt_list, i)#platform
        nonfunc_gpl[[platform_char]] = platform
        next
    }
  }
  
  gene_profiles = gene_profiles[ ,c("Symbol","VALUE")]
  gene_profiles = gene_profiles[!(gene_profiles$Symbol == "---" | gene_profiles$Symbol == ""), ]
  names(gene_profiles)[names(gene_profiles) == "VALUE"] = i
  
  if (count == 0){
    final_res = gene_profiles
      
  } else if (count == 50){
      final_res = final_res%>%full_join(gene_profiles, by="Symbol")
      final_res = final_res%>%group_by(Symbol)%>%summarise_all("mean")
      
      fwrite(final_res, paste0(i,"_" ,Sys.time(),"_" ,".csv"))# i is the last gsm included in write file
      count = 0
    
  } else {
    final_res = final_res%>%full_join(gene_profiles, by="Symbol")
    final_res = final_res%>%group_by(Symbol)%>%summarise_all("mean") #aggregate rows containing duplicate Symbol elements
  }
  
  count = count + 1
}
fwrite(final_res, paste0(i,"_" ,Sys.time(),"_" ,".csv"))

In [ ]:
#b[!(grepl('-Dec', b$Symbol) | grepl('-Mar', b$Symbol)), ]

In [3]:
#w = read.csv('GSM1573169_2020-11-05 02:11:26_.csv')
#x = read.csv('GSM1422010_2020-11-05 03:01:59_.csv')
#y = read.csv('GSM1569196_2020-11-05 04:44:20_.csv')
z = read.csv('GSM1519747_2020-11-05 05:17:11_.csv')

res = setdiff(results$REPOSITORY_ACCESSION, colnames(z))

In [4]:
count = 0
gpl_dict = dict()
#nonfunc_gpl = dict()### non-functional GPL ID dict or list for colnames
gsm_vec = vector()
gpl_vec = vector()
nonfunc_gsm = vector()

In [5]:
for(i in res) {
    ### LOADING TABLES
    GSM = getGEO(i, getGPL=FALSE)
    
    if(length(colnames(GSM@dataTable@table)) == 0){
        empty_gsm = c(nonfunc_gsm, i)#empty gsm ids
        next
    }
    
    genex = GSM@dataTable@table[,1:2]# it exists, load it
    platform_char = GSM@header$platform_id
  
    ### check if platform id already loaded
    if(platform_char %in% gpl_dict$keys()){
        ### at this point the platform table should only have cols ID and Symbol
        gene_profiles = genex%>%inner_join(gpl_dict[[platform_char]], by=c("ID_REF"="ID"))
    
    } else { ### we have a unique platform GPL ID to add to the dict
        ### load unique GPL
        platform_obj = getGEO(platform_char, getGPL=FALSE)
        platform = platform_obj@dataTable@table
        
        if("ID" %in% colnames(platform) && str_contains(colnames(platform), "Symbol")){
            names(platform)[grepl("Symbol", names(platform))] = "Symbol"
            platform = platform[,c("ID","Symbol")]

            gpl_dict[[platform_char]] = platform ## add platform table to dict
            gene_profiles = genex%>%inner_join(platform, by=c("ID_REF"="ID"))
      
        } else {
            ### store non-functional GPL w/ platform table or colnames in dict
            gsm_vec = c(gsm_vec, i)# gsm list at platform id level
            gpl_vec = c(gpl_vec, platform_char)# corresponding gpl list
            next
        }
    }
  
    gene_profiles = gene_profiles[ ,c("Symbol","VALUE")]
    gene_profiles = gene_profiles[!(gene_profiles$Symbol == "---" | gene_profiles$Symbol == ""), ]
    names(gene_profiles)[names(gene_profiles) == "VALUE"] = i
      
    if (count == 0 && exists('gene_profiles')){
        final_res1 = gene_profiles
    
    } else {
        final_res1 = final_res1%>%full_join(gene_profiles, by="Symbol")
        final_res1 = final_res1%>%group_by(Symbol)%>%summarise_all("mean") #aggregate rows containing duplicate Symbol elements
        
        count = count + 1 # count needs to be located in this else statement 
                          # otherwise it will increment and a full join will 
                          # proceed without a table
    }
}

if(exists('final_res1')){
    fwrite(final_res1, paste0(i,"_" ,Sys.time(),"_" ,".csv"))
}

fwrite(data.frame(gpl_vec, gsm_vec), paste0("nonfunctional_gpl_gsm_dataframe" ,".csv"))# non func gpl_vec, gsm_vec
#fwrite(data.frame(nonfunc_gsm), paste0("empty_gsm_data_table" ,".csv"))#empty gsm data table

File stored at: 
/tmp/RtmppJkQv0/GSM1008332.soft
File stored at: 
/tmp/RtmppJkQv0/GSM1008368.soft
File stored at: 
/tmp/RtmppJkQv0/GSM1008377.soft
File stored at: 
/tmp/RtmppJkQv0/GSM1008401.soft
File stored at: 
/tmp/RtmppJkQv0/GSM1008329.soft
File stored at: 
/tmp/RtmppJkQv0/GSM1008311.soft
File stored at: 
/tmp/RtmppJkQv0/GSM1008312.soft
File stored at: 
/tmp/RtmppJkQv0/GSM1008313.soft
File stored at: 
/tmp/RtmppJkQv0/GSM1008315.soft
File stored at: 
/tmp/RtmppJkQv0/GSM1008316.soft
File stored at: 
/tmp/RtmppJkQv0/GSM1008317.soft
File stored at: 
/tmp/RtmppJkQv0/GSM1008320.soft
File stored at: 
/tmp/RtmppJkQv0/GSM1008321.soft
File stored at: 
/tmp/RtmppJkQv0/GSM1008323.soft
File stored at: 
/tmp/RtmppJkQv0/GSM1008324.soft
File stored at: 
/tmp/RtmppJkQv0/GSM1008326.soft
File stored at: 
/tmp/RtmppJkQv0/GSM1008327.soft
File stored at: 
/tmp/RtmppJkQv0/GSM1008328.soft
File stored at: 
/tmp/RtmppJkQv0/GSM1008330.soft
File stored at: 
/tmp/RtmppJkQv0/GSM1008331.soft
File stored at: 
/tm

In [ ]:
data.frame(nonfunc_gsm)

In [ ]:
nonfunc_gpl$keys()

In [ ]:
nonfunc_gpl$values()[1]

In [ ]:
#names(dt_list)

In [ ]:
length(unique(colnames(x)))

In [ ]:
head(x)

In [ ]:
length(gsm_list)

In [ ]:
length(unique(gpl_list))

In [ ]:
head(final_res1)

In [ ]:
gene_profiles

In [ ]:
getwd()

In [ ]:
list.files()[grepl('GSM', list.files())]

In [ ]:
colnames(w)

In [ ]:
w = w[!(grepl('-Dec', w$Symbol) | grepl('-Mar', w$Symbol)), ]
x = x[!(grepl('-Dec', x$Symbol) | grepl('-Mar', x$Symbol)), ]
y = y[!(grepl('-Dec', y$Symbol) | grepl('-Mar', y$Symbol)), ]
z = z[!(grepl('-Dec', z$Symbol) | grepl('-Mar', z$Symbol)), ]

In [ ]:
length(intersect(colnames(z), colnames(w)))# all gsm files contain duplicate columns 

In [ ]:
length(sort(colnames(w)))

In [ ]:
length(colnam)

In [ ]:
#unique(union(colnames(w), colnames(x)))
head(z)

In [ ]:
length(colnames(w))

In [ ]:
diffw = setdiff(colnames(z), colnames(x))
print(length(diffw))

In [ ]:
#w[,diffw]

In [ ]:
count = 0

In [ ]:
for(f in list.files()[grepl('GSM', list.files())]){
    df = read.csv(f)
    df = df[!(grepl('-Dec', df$Symbol) | grepl('-Mar', df$Symbol)), ]
    
    if (count == 0){
        final_res = df
    
    } else {
        diff = setdiff(colnames(df), colnames(final_res))# only unique cols from df 
        final_res = final_res%>%full_join(diff, by='Symbol')
        final_res = final_res%>%group_by(Symbol)%>%summarise_all('mean')
    }
    
    count = count + 1
}# 25-30 mins

In [ ]:
# x('GSM1519747_2020-11-05 05:17:11_.csv') is the only good file// poorly placed cell// rearrange notebook

df = x[!(grepl('-Dec', x$Symbol) | grepl('-Mar', x$Symbol)), ]

final_res = df%>%group_by(Symbol)%>%summarise_all('mean')
meanSymb = final_res%>%pivot_longer(cols = -Symbol)%>%pivot_wider(names_from = Symbol, values_from = value)

In [ ]:
head(meanSymb)

In [ ]:
head(df)

In [ ]:
length(unique(colnames(final_res)))

In [ ]:
#df[!(grepl('-Dec', df$Symbol) | grepl('-Mar', df$Symbol)), ]

In [ ]:
print(dim(final_res))

In [ ]:
print(head(final_res))

In [ ]:
meanSymb = final_res%>%pivot_longer(cols = -Symbol)%>%pivot_wider(names_from = Symbol, values_from = value)

In [ ]:
fwrite(final_res, paste0(i,"_" ,Sys.time(),"_" ,".csv"))# Nov 6 should be last data set

In [ ]:
head(final_res)

In [ ]:
#fwrite(genex, "_.csv") works but not in the for loop bc it can't find object
count = 0
gpl_dict = dict()
nonfunc_gpl = dict()### non-functional GPL ID dict or list for colnames
nonfunc_gsm = vector()

In [ ]:
for(i in setdiff(res, nonfunc_gsm)) {#res
    ### LOADING TABLES
    GSM = getGEO(i, getGPL=FALSE)
    
    if(length(colnames(GSM@dataTable@table)) == 0){
        nonfunc_gsm = c(nonfunc_gsm, i)
        next
    }
    
    genex = GSM@dataTable@table[,1:2]# it exists, load it
    platform_char = GSM@header$platform_id
  
    ### check if platform id already loaded
    if(platform_char %in% gpl_dict$keys()){
        ### at this point the platform table should only have cols ID and Symbol
        gene_profiles = genex%>%inner_join(gpl_dict[[platform_char]], by=c("ID_REF"="ID"))
    
    } else { ### we have a unique platform GPL ID to add to the dict
        ### load unique GPL
        platform_obj = getGEO(platform_char, getGPL=FALSE)
        platform = platform_obj@dataTable@table
        
        if("ID" %in% colnames(platform) && str_contains(colnames(platform), "Symbol")){
            names(platform)[grepl("Symbol", names(platform))] = "Symbol"
            platform = platform[,c("ID","Symbol")]

            gpl_dict[[platform_char]] = platform ## add platform table to dict
            gene_profiles = genex%>%inner_join(platform, by=c("ID_REF"="ID"))
      
        } else {
            ### store non-functional GPL w/ platform table or colnames in dict
            nonfunc_gpl[[platform_char]] = platform# one gpl is responsible for all bad gsm, 
                                                   # so need to bag into list b/c one table 
                                                   # is overwriting all other tables, expecting
                                                   # one value per key 
            next
        }
    }
  
    gene_profiles = gene_profiles[ ,c("Symbol","VALUE")]
    gene_profiles = gene_profiles[!(gene_profiles$Symbol == "---" | gene_profiles$Symbol == ""), ]
    names(gene_profiles)[names(gene_profiles) == "VALUE"] = i
  
    if (count == 0){
        final_res1 = gene_profiles
        
        #print(head(final_res1))
    
    } else if (count == 50){
        final_res1 = final_res1%>%full_join(gene_profiles, by="Symbol")
        final_res1 = final_res1%>%group_by(Symbol)%>%summarise_all("mean")
        
        #print(head(final_res1))
        
        fwrite(final_res1, paste0(i,"_" ,Sys.time(),"_" ,".csv"))# i is the last gsm included in write file
        count = 0
    
    } else {
        final_res1 = final_res1%>%full_join(gene_profiles, by="Symbol")
        final_res1 = final_res1%>%group_by(Symbol)%>%summarise_all("mean") #aggregate rows containing duplicate Symbol elements
        
        count = count + 1 # count needs to be here otherwise it will increment and a full join will occur without a table
    }
}
#fwrite(final_res1, paste0(i,"_" ,Sys.time(),"_" ,".csv"))

In [ ]:
#GSM1132316
#res -- made it through all GSM without creating gene_profiles, final_res object, writing files
length(nonfunc_gsm)

In [ ]:
length(res)

In [ ]:
length(setdiff(res, nonfunc_gsm))